In [2]:
!git clone https://github.com/Deep-MI/FastSurfer.git

Cloning into 'FastSurfer'...
remote: Enumerating objects: 10331, done.
remote: Counting objects: 100% (2266/2266), done.
remote: Compressing objects: 100% (483/483), done.
remote: Total 10331 (delta 2001), reused 1791 (delta 1779), pack-reused 8065 (from 3)
Receiving objects: 100% (10331/10331), 86.33 MiB | 36.43 MiB/s, done.
Resolving deltas: 100% (7484/7484), done.


In [3]:
%cd FastSurfer

/kaggle/working/FastSurfer


In [4]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu126
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.2/101.2 kB 2.1 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 44.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.9/164.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 79.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install torch torchvision torchaudio nibabel numpy scipy scikit-image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [6]:
!mkdir -p ckpts/FastSurferCNN

In [7]:
!wget -O ckpts/FastSurferCNN/FastSurferCNN_weights.pt \
https://dl.dropboxusercontent.com/scl/fi/7t90wl6mglx4nxp5aibj3/FastSurferCNN_weights.pt?rlkey=2to4x1cjqgicd9g4nfd6xd8hh&dl=1

In [8]:
!pip install yacs

  Using cached yacs-0.1.8-py3-none-any.whl.metadata (639 bytes)
Using cached yacs-0.1.8-py3-none-any.whl (14 kB)


In [9]:
import sys
sys.path.append("/kaggle/working/FastSurfer")

In [11]:
import nibabel as nib
import numpy as np
import os
import csv
import subprocess
import sys

# Define the input directory containing NIfTI files
input_dir = '/kaggle/input/skull-stripped-to-csv'  # Directory containing your NIfTI files

# Create a list to hold the results
results = []

# Walk through the directory and process each NIfTI file
for root, dirs, files in os.walk(input_dir):
    for file in files:
        if file.endswith('.nii') or file.endswith('.nii.gz'):  # Process only NIfTI files
            t1_file = os.path.join(root, file)
            sid = os.path.basename(t1_file).split('_')[0]  # Extract the subject ID (e.g., I9856)

            # Construct the path to the segmentation file
            seg_path = f"./output/{sid}/mri/aparc.DKTatlas+aseg.deep.mgz"  # Path to the segmentation output
            
            print(f"Processing subject: {sid}")
            print(f"Looking for segmentation file at: {seg_path}")

            # Check if the segmentation file exists
            if not os.path.exists(seg_path):
                print(f"Segmentation file for {sid} not found. Running FastSurfer prediction...")
                
                # Construct the FastSurfer command
                fastsurfer_command = [
                    "python3",
                    "/kaggle/working/FastSurfer/FastSurferCNN/run_prediction.py",
                    "--t1", t1_file,
                    "--sid", sid,
                    "--sd", "./output",
                    "--batch_size", "4"
                ]
                
                # Set the environment variable PYTHONPATH and run the command
                env = os.environ.copy()
                env["PYTHONPATH"] = "/kaggle/working/FastSurfer"

                try:
                    subprocess.run(fastsurfer_command, env=env, check=True)
                    print(f"FastSurfer prediction completed for {sid}")
                except subprocess.CalledProcessError as e:
                    print(f"Error running FastSurfer for {sid}: {e}")
                    continue

            # Load the segmentation data (either from DKT or aseg)
            try:
                print(f"Attempting to load segmentation file: {seg_path}")
                seg = nib.load(seg_path)
                seg_data = seg.get_fdata()
                voxel_volume = np.prod(seg.header.get_zooms())  # Calculate voxel volume
                
                # Calculate volumes for left and right hippocampus
                left_hip = np.sum(seg_data == 17) * voxel_volume
                right_hip = np.sum(seg_data == 53) * voxel_volume
                
                # Store the result (subject ID and hippocampal volumes)
                results.append([sid, left_hip, right_hip])
                print(f"Left Hippocampus: {left_hip:.2f} mm³, Right Hippocampus: {right_hip:.2f} mm³")
            except FileNotFoundError:
                print(f"Segmentation file for {sid} still not found after running FastSurfer. Skipping...")
                continue

# Define the CSV file path
csv_file = "./hippocampal_volumes.csv"

# Write the results to a CSV file
with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['SubjectID', 'Left Hippocampus Volume (mm³)', 'Right Hippocampus Volume (mm³)'])
    writer.writerows(results)

print(f"Results saved to {csv_file}")


Processing subject: I335530
Looking for segmentation file at: ./output/I335530/mri/aparc.DKTatlas+aseg.deep.mgz
Attempting to load segmentation file: ./output/I335530/mri/aparc.DKTatlas+aseg.deep.mgz
Left Hippocampus: 4378.00 mm³, Right Hippocampus: 4391.00 mm³
Processing subject: I401935
Looking for segmentation file at: ./output/I401935/mri/aparc.DKTatlas+aseg.deep.mgz
Attempting to load segmentation file: ./output/I401935/mri/aparc.DKTatlas+aseg.deep.mgz
Left Hippocampus: 3097.00 mm³, Right Hippocampus: 3195.00 mm³
Processing subject: I432170
Looking for segmentation file at: ./output/I432170/mri/aparc.DKTatlas+aseg.deep.mgz
Attempting to load segmentation file: ./output/I432170/mri/aparc.DKTatlas+aseg.deep.mgz
Left Hippocampus: 4198.00 mm³, Right Hippocampus: 4411.00 mm³
Processing subject: I328605
Looking for segmentation file at: ./output/I328605/mri/aparc.DKTatlas+aseg.deep.mgz
Attempting to load segmentation file: ./output/I328605/mri/aparc.DKTatlas+aseg.deep.mgz
Left Hippocamp

100%|██████████| 64/64 [00:06<00:00, 10.06batch/s]
Inference on 64 batches for coronal successful
100%|██████████| 64/64 [00:05<00:00, 11.70batch/s]
Inference on 64 batches for sagittal successful
100%|██████████| 64/64 [00:05<00:00, 11.08batch/s]
Inference on 64 batches for axial successful


Creating dilated mask ...
Frontal region special treatment:  18220
  Found 1 connected component(s)!
Reducing to aseg ...
FlipWM: rh 0 and lh 0 flipped.
Checking total volume ...
Voxel size in mm3: 1.0
Total segmentation volume in liter: 1.04
FastSurfer prediction completed for I742548
Attempting to load segmentation file: ./output/I742548/mri/aparc.DKTatlas+aseg.deep.mgz
Left Hippocampus: 4107.00 mm³, Right Hippocampus: 4032.00 mm³
Processing subject: I240141
Looking for segmentation file at: ./output/I240141/mri/aparc.DKTatlas+aseg.deep.mgz
Segmentation file for I240141 not found. Running FastSurfer prediction...
Input:    min: 0  max: 425
rescale:  min: 0.0  max: 370.175  scale: 0.6888633754305395
Output:   min: 0.0  max: 255.0


100%|██████████| 64/64 [00:06<00:00,  9.96batch/s]
Inference on 64 batches for coronal successful
100%|██████████| 64/64 [00:05<00:00, 11.61batch/s]
Inference on 64 batches for sagittal successful
100%|██████████| 64/64 [00:05<00:00, 11.00batch/s]
Inference on 64 batches for axial successful


Creating dilated mask ...
Frontal region special treatment:  18998
  Found 1 connected component(s)!
Reducing to aseg ...
FlipWM: rh 0 and lh 0 flipped.
Checking total volume ...
Voxel size in mm3: 1.0
Total segmentation volume in liter: 1.1
FastSurfer prediction completed for I240141
Attempting to load segmentation file: ./output/I240141/mri/aparc.DKTatlas+aseg.deep.mgz
Left Hippocampus: 4158.00 mm³, Right Hippocampus: 3800.00 mm³
Processing subject: I366391
Looking for segmentation file at: ./output/I366391/mri/aparc.DKTatlas+aseg.deep.mgz
Segmentation file for I366391 not found. Running FastSurfer prediction...
Input:    min: 0  max: 694
rescale:  min: 0.0  max: 464.28599999999994  scale: 0.5492304312428116
Output:   min: 0.0  max: 255.0


...

Creating dilated mask ...
Frontal region special treatment:  21873
  Found 1 connected component(s)!
Reducing to aseg ...
FlipWM: rh 0 and lh 0 flipped.
Checking total volume ...
Voxel size in mm3: 1.0
Total segmentation volume in liter: 1.18
FastSurfer prediction completed for I325610
Attempting to load segmentation file: ./output/I325610/mri/aparc.DKTatlas+aseg.deep.mgz
Left Hippocampus: 3566.00 mm³, Right Hippocampus: 3703.00 mm³
Processing subject: I29604
Looking for segmentation file at: ./output/I29604/mri/aparc.DKTatlas+aseg.deep.mgz
Segmentation file for I29604 not found. Running FastSurfer prediction...
Input:    min: 0  max: 641
rescale:  min: 0.0  max: 480.10900000000004  scale: 0.531129389367831
Output:   min: 0.0  max: 255.0


100%|██████████| 64/64 [00:06<00:00,  9.93batch/s]
Inference on 64 batches for coronal successful
100%|██████████| 64/64 [00:05<00:00, 11.39batch/s]
Inference on 64 batches for sagittal successful
100%|██████████| 64/64 [00:06<00:00, 10.52batch/s]
Inference on 64 batches for axial successful


Creating dilated mask ...
Frontal region special treatment:  19177
  Found 1 connected component(s)!
Reducing to aseg ...
FlipWM: rh 0 and lh 0 flipped.
Checking total volume ...
Voxel size in mm3: 1.0
Total segmentation volume in liter: 1.1
FastSurfer prediction completed for I29604
Attempting to load segmentation file: ./output/I29604/mri/aparc.DKTatlas+aseg.deep.mgz
Left Hippocampus: 3761.00 mm³, Right Hippocampus: 4028.00 mm³
Processing subject: I129821
Looking for segmentation file at: ./output/I129821/mri/aparc.DKTatlas+aseg.deep.mgz
Segmentation file for I129821 not found. Running FastSurfer prediction...
Input:    min: 0  max: 457
rescale:  min: 0.0  max: 413.12800000000004  scale: 0.6172421138242868
Output:   min: 0.0  max: 255.0


100%|██████████| 64/64 [00:06<00:00,  9.42batch/s]
Inference on 64 batches for coronal successful
100%|██████████| 64/64 [00:05<00:00, 11.34batch/s]
Inference on 64 batches for sagittal successful
100%|██████████| 64/64 [00:05<00:00, 10.87batch/s]
Inference on 64 batches for axial successful


Creating dilated mask ...
Frontal region special treatment:  18893
  Found 1 connected component(s)!
Reducing to aseg ...
FlipWM: rh 0 and lh 0 flipped.
Checking total volume ...
Voxel size in mm3: 1.0
Total segmentation volume in liter: 1.1
FastSurfer prediction completed for I129821
Attempting to load segmentation file: ./output/I129821/mri/aparc.DKTatlas+aseg.deep.mgz
Left Hippocampus: 3942.00 mm³, Right Hippocampus: 4022.00 mm³
Results saved to ./hippocampal_volumes.csv
